In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%run 1.tokenizer.ipynb

tokenizer

In [2]:
%run 2.dataset.ipynb


def f(data):
    text = [i['text'] for i in data]
    label = [i['label'] for i in data]

    data = tokenizer(text, device=device)
    data['labels'] = torch.FloatTensor(label).to(device)

    return data


loader = get_loader(f, negative_label=True, with_answer=True)

len(loader), next(iter(loader))

(62500,
 {'input_ids': tensor([[ 0,  5,  6, 10,  6, 14,  7,  9, 11,  8, 18, 12, 12,  8,  5,  1,  2],
          [ 0,  9,  8, 13,  8, 14, 12, 13,  9,  4, 18,  5,  8,  8,  8,  8,  1],
          [ 0,  5, 10,  5,  6, 14,  5,  7,  8, 10, 18,  6, 13,  9, 12,  1,  2],
          [ 0, 11,  8,  6, 10, 14, 12, 12,  7, 13, 18,  5, 10,  6, 10,  9,  1],
          [ 0,  5, 10, 11, 10, 14, 10, 12,  7,  5, 18, 10,  8,  8, 12,  1,  2],
          [ 0, 13,  9,  7,  9, 14, 12, 13, 12,  5, 18,  5,  6,  9, 10, 11,  1],
          [ 0,  8, 11,  5, 12, 14,  7,  6,  7, 18,  9,  4,  8,  5,  1,  2,  2],
          [ 0,  8,  9,  8,  6, 14, 13,  8, 13, 10, 18,  5,  8,  4,  7, 12,  1],
          [ 0,  5, 13, 10,  9, 14,  5, 12,  6, 11, 18,  5,  4,  9,  8,  4,  1],
          [ 0, 13,  9,  5,  9, 14,  8, 10, 10, 13, 18,  5,  8,  5, 12,  8,  1],
          [ 0,  5, 11, 12,  6, 14,  7,  8, 11,  8, 18,  9,  6,  9, 10,  1,  2],
          [ 0,  7, 12, 12, 13, 14,  9,  7,  6,  4, 18, 13,  6,  4, 13,  1,  2],
          [ 0, 10, 

In [3]:
%run 3.model.ipynb

model_critic = Gemma3Critic(len(tokenizer), tokenizer.pad_token_id,
                            tokenizer.eos_token_id).to(device)

sum(i.numel() for i in model_critic.parameters()) / 1_0000_0000

2.01422848

In [4]:
optimizer = torch.optim.Adam(model_critic.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                              base_lr=1e-5,
                                              max_lr=1e-4,
                                              step_size_up=1000)

for i, data in enumerate(loader):
    loss, logits = model_critic(**data)
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    if i % 1000 == 0:
        logits = (logits > 0.5).squeeze(1).long()
        acc = (logits == data['labels'].long()).sum() / len(data['labels'])
        print(i, len(loader), loss.item(), acc.item())

torch.save(model_critic.to('cpu'), 'model/critic')

0 62500 0.6568640470504761 0.625
1000 62500 0.22538667917251587 0.5625
2000 62500 0.15205013751983643 0.75
3000 62500 0.2514706254005432 0.625
4000 62500 0.13919974863529205 0.8125
5000 62500 0.17328010499477386 0.75
6000 62500 0.1473165601491928 0.8125
7000 62500 0.01866268552839756 1.0
8000 62500 0.09422843158245087 0.875
9000 62500 0.2106718122959137 0.75
10000 62500 0.15441328287124634 0.8125
11000 62500 0.04718015715479851 0.9375
12000 62500 0.03505060821771622 0.9375
13000 62500 0.05470049008727074 1.0
14000 62500 0.05251363664865494 0.9375
15000 62500 0.12895986437797546 0.8125
16000 62500 0.08369071781635284 0.8125
17000 62500 0.15602001547813416 0.8125
18000 62500 0.04057002067565918 0.9375
19000 62500 0.04003428667783737 1.0
20000 62500 0.06784521788358688 0.875
21000 62500 0.06001725792884827 0.875
22000 62500 0.0022971846628934145 1.0
23000 62500 0.06056694686412811 0.9375
24000 62500 0.05474509671330452 0.9375
25000 62500 0.022833609953522682 1.0
26000 62500 0.057579100131